In [4]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import tensorflow as tf
import psycopg2
from sqlalchemy import create_engine


Housing Dataset
---------------


**Dataset characteristics:**

    :Number of instances: 54,670

    :Number of Attributes: 8

    :Attribute Information:

        -'Estimated_Median_Household_Income' Median household income for the year for the city
        -'2021_estimated_population' estimated population for the city in 2021
        -'2020_census' population by census in 2020
        -'Median_Taxes' Median taxes paid for the household in the city
        -'Median_Rent' Median rent paid in the city for each year
        -'AnnualAverageRate30Year' Average mortgage rate for city for 30 year mortgages
        -'Median_Monthly_Income' Median income divided by 12
        -'Monthly_Affordability_Limit' based on 30% of median monthly income

    :Missing Attribute Values:

This dataset was obtained from zillow research data and US census data from 2011 to 2021.

The target variable is 'affordability_home_30yr_Payment_20_Perc_Down'



In [13]:
from config import engine_url

# Create an engine instance

alchemyEngine   = create_engine(engine_url, pool_recycle=3600);

 

# Connect to PostgreSQL server

dbConnection    = alchemyEngine.connect();

 

# Read data from PostgreSQL database table and load into a DataFrame instance

df       = pd.read_sql("select * from \"master_data_set\"", dbConnection);

 

pd.set_option('display.expand_frame_repr', False);

 

# Print the DataFrame

print(df);

 

# Close the database connection

dbConnection.close();

     Identifier  City_Rank_by_Population(2021)       State State_abbreviation         City Observation_Date  Single_Family_Median_Typical_Home_Value  One_Bedroom_Median_Typical_Home_Value  Two_Bedroom_Median_Typical_Home_Value  Three_Bedroom_Median_Typical_Home_Value  ...  affordability_3br_30yr_Payment_10_Perc_Down  affordability_3br_30yr_Payment_20_Perc_Down  affordability_4br_15yr_Payment_10_Perc_Down  affordability_4br_15yr_Payment_20_Perc_Down  affordability_4br_30yr_Payment_10_Perc_Down  affordability_4br_30yr_Payment_20_Perc_Down  affordability_5_plus_br_15yr_Payment_10_Perc_Down  affordability_5_plus_br_15yr_Payment_20_Perc_Down  affordability_5_plus_br_30yr_Payment_10_Perc_Down  affordability_5_plus_br_30yr_Payment_20_Perc_Down
0        201101                              1    New York                 NY     New York       2011-01-01                                 460824.0                               419555.5                               549450.5                           

In [14]:
df

,Identifier,City_Rank_by_Population(2021),State,State_abbreviation,City,Observation_Date,Single_Family_Median_Typical_Home_Value,One_Bedroom_Median_Typical_Home_Value,Two_Bedroom_Median_Typical_Home_Value,Three_Bedroom_Median_Typical_Home_Value,...,affordability_3br_30yr_Payment_10_Perc_Down,affordability_3br_30yr_Payment_20_Perc_Down,affordability_4br_15yr_Payment_10_Perc_Down,affordability_4br_15yr_Payment_20_Perc_Down,affordability_4br_30yr_Payment_10_Perc_Down,affordability_4br_30yr_Payment_20_Perc_Down,affordability_5_plus_br_15yr_Payment_10_Perc_Down,affordability_5_plus_br_15yr_Payment_20_Perc_Down,affordability_5_plus_br_30yr_Payment_10_Perc_Down,affordability_5_plus_br_30yr_Payment_20_Perc_Down
0,201101,1,New York,NY,New York,2011-01-01,460824.0,419555.5,549450.5,455413.0,...,0.60,0.67,0.36,0.40,0.51,0.58,0.29,0.32,0.41,0.46
1,201201,1,New York,NY,New York,2012-01-01,453166.5,421581.5,549293.0,448447.0,...,0.69,0.77,0.39,0.44,0.59,0.66,0.31,0.35,0.47,0.53
2,201301,1,New York,NY,New York,2013-01-01,467125.5,445182.5,579800.5,461446.5,...,0.66,0.74,0.38,0.43,0.56,0.63,0.30,0.34,0.44,0.50
3,201401,1,New York,NY,New York,2014-01-01,499898.0,487199.5,634870.5,493995.0,...,0.61,0.69,0.36,0.40,0.51,0.58,0.28,0.31,0.40,0.45
4,201501,1,New York,NY,New York,2015-01-01,526189.0,536285.0,689511.5,522588.5,...,0.63,0.71,0.36,0.40,0.53,0.59,0.27,0.30,0.40,0.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,201775,75,New Jersey,NJ,Jersey City,2017-01-01,408525.0,471067.5,537922.5,458794.5,...,0.84,0.95,0.59,0.66,0.87,0.98,0.55,0.61,0.81,0.91
777,201875,75,New Jersey,NJ,Jersey City,2018-01-01,459463.0,503269.5,568473.0,501645.0,...,0.79,0.89,0.55,0.62,0.80,0.90,0.50,0.57,0.73,0.82
778,201975,75,New Jersey,NJ,Jersey City,2019-01-01,489001.0,490562.5,566059.0,523939.5,...,0.91,1.03,0.60,0.68,0.90,1.02,0.56,0.62,0.83,0.94
779,202075,75,New Jersey,NJ,Jersey City,2020-01-01,514131.5,487710.5,570039.5,541306.0,...,0.92,1.03,0.58,0.65,0.90,1.02,0.52,0.59,0.82,0.92


In [15]:
df['affordability_home_30yr_Payment_20_Perc_Down'].info()


<class 'pandas.core.series.Series'>
RangeIndex: 781 entries, 0 to 780
Series name: affordability_home_30yr_Payment_20_Perc_Down
Non-Null Count  Dtype  
--------------  -----  
781 non-null    float64
dtypes: float64(1)
memory usage: 6.2 KB


In [16]:
columns_list = df.columns.to_list()

target_list = columns_list[45:]

target_list

['affordability_rent',
 'affordability_home_15yr_Payment_10_Perc_Down',
 'affordability_home_15yr_Payment_20_Perc_Down',
 'affordability_home_30yr_Payment_10_Perc_Down',
 'affordability_home_30yr_Payment_20_Perc_Down',
 'affordability_1br_15yr_Payment_10_Perc_Down',
 'affordability_1br_15yr_Payment_20_Perc_Down',
 'affordability_1br_30yr_Payment_10_Perc_Down',
 'affordability_1br_30yr_Payment_20_Perc_Down',
 'affordability_2br_15yr_Payment_10_Perc_Down',
 'affordability_2br_15yr_Payment_20_Perc_Down',
 'affordability_2br_30yr_Payment_10_Perc_Down',
 'affordability_2br_30yr_Payment_20_Perc_Down',
 'affordability_3br_15yr_Payment_10_Perc_Down',
 'affordability_3br_15yr_Payment_20_Perc_Down',
 'affordability_3br_30yr_Payment_10_Perc_Down',
 'affordability_3br_30yr_Payment_20_Perc_Down',
 'affordability_4br_15yr_Payment_10_Perc_Down',
 'affordability_4br_15yr_Payment_20_Perc_Down',
 'affordability_4br_30yr_Payment_10_Perc_Down',
 'affordability_4br_30yr_Payment_20_Perc_Down',
 'affordabili

In [18]:
df[target_list] = np.where(df[target_list]>1.00,True, False)
df

,Identifier,City_Rank_by_Population(2021),State,State_abbreviation,City,Observation_Date,Single_Family_Median_Typical_Home_Value,One_Bedroom_Median_Typical_Home_Value,Two_Bedroom_Median_Typical_Home_Value,Three_Bedroom_Median_Typical_Home_Value,...,affordability_3br_30yr_Payment_10_Perc_Down,affordability_3br_30yr_Payment_20_Perc_Down,affordability_4br_15yr_Payment_10_Perc_Down,affordability_4br_15yr_Payment_20_Perc_Down,affordability_4br_30yr_Payment_10_Perc_Down,affordability_4br_30yr_Payment_20_Perc_Down,affordability_5_plus_br_15yr_Payment_10_Perc_Down,affordability_5_plus_br_15yr_Payment_20_Perc_Down,affordability_5_plus_br_30yr_Payment_10_Perc_Down,affordability_5_plus_br_30yr_Payment_20_Perc_Down
0,201101,1,New York,NY,New York,2011-01-01,460824.0,419555.5,549450.5,455413.0,...,False,False,False,False,False,False,False,False,False,False
1,201201,1,New York,NY,New York,2012-01-01,453166.5,421581.5,549293.0,448447.0,...,False,False,False,False,False,False,False,False,False,False
2,201301,1,New York,NY,New York,2013-01-01,467125.5,445182.5,579800.5,461446.5,...,False,False,False,False,False,False,False,False,False,False
3,201401,1,New York,NY,New York,2014-01-01,499898.0,487199.5,634870.5,493995.0,...,False,False,False,False,False,False,False,False,False,False
4,201501,1,New York,NY,New York,2015-01-01,526189.0,536285.0,689511.5,522588.5,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,201775,75,New Jersey,NJ,Jersey City,2017-01-01,408525.0,471067.5,537922.5,458794.5,...,False,False,False,False,False,False,False,False,False,False
777,201875,75,New Jersey,NJ,Jersey City,2018-01-01,459463.0,503269.5,568473.0,501645.0,...,False,False,False,False,False,False,False,False,False,False
778,201975,75,New Jersey,NJ,Jersey City,2019-01-01,489001.0,490562.5,566059.0,523939.5,...,False,True,False,False,False,True,False,False,False,False
779,202075,75,New Jersey,NJ,Jersey City,2020-01-01,514131.5,487710.5,570039.5,541306.0,...,False,True,False,False,False,True,False,False,False,False


In [19]:
x_columns = ['Estimated_Median_Household_Income', '2021_estimated_population', '2020_census', 'Median_Taxes', 'Median_Rent', 'AnnualAverageRate30Year','Median_Monthly_Income','Monthly_Affordability_Limit'] 
# 'home_30yr_Payment_20_Perc_Down', '1br_30yr_Payment_20_Perc_Down', '2br_30yr_Payment_20_Perc_Down', 
# '3br_30yr_Payment_20_Perc_Down', '4br_30yr_Payment_20_Perc_Down', '5_plus_br_30yr_Payment_20_Perc_Down', 'Median_Monthly_Income', 'Monthly_Affordability_Limit'



X = df[x_columns]
# y=df['affordability_home_30yr_Payment_20_Perc_Down']

In [79]:
corr_df = X.corr()
corr_df

,Estimated_Median_Household_Income,2021_estimated_population,2020_census,Median_Taxes,Median_Rent,AnnualAverageRate30Year,Median_Monthly_Income,Monthly_Affordability_Limit
Estimated_Median_Household_Income,1.000000,1.607488e-02,1.861983e-02,0.664606,0.871692,-2.140559e-01,1.000000,1.000000
2021_estimated_population,0.016075,1.000000e+00,9.998610e-01,0.158370,0.124975,-6.951612e-16,0.016075,0.016075
2020_census,0.018620,9.998610e-01,1.000000e+00,0.161385,0.127955,-6.742022e-16,0.018620,0.018620
Median_Taxes,0.664606,1.583703e-01,1.613845e-01,1.000000,0.715111,-1.270912e-01,0.664606,0.664606
Median_Rent,0.871692,1.249748e-01,1.279552e-01,0.715111,1.000000,-2.154704e-01,0.871692,0.871692
AnnualAverageRate30Year,-0.214056,-6.951612e-16,-6.742022e-16,-0.127091,-0.215470,1.000000e+00,-0.214056,-0.214056
Median_Monthly_Income,1.000000,1.607497e-02,1.861993e-02,0.664606,0.871692,-2.140559e-01,1.000000,1.000000
Monthly_Affordability_Limit,1.000000,1.607540e-02,1.862035e-02,0.664606,0.871692,-2.140559e-01,1.000000,1.000000


In [20]:
for target in target_list:
    y=df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
    scaler = StandardScaler().fit(X_train)
    X_train_scaled = scaler.transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    # model = LinearRegression()

    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
    nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the Sequential model together and customize metrics
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


    # Train the model
    fit_model = nn_model.fit(X_train_scaled, y_train, epochs=100)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn_model.evaluate(X_test_scaled,y_test,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
19/19 [==============================] - 0s 887us/step - loss: 0.8617 - accuracy: 0.2940
Epoch 2/100
19/19 [==============================] - 0s 1ms/step - loss: 0.6968 - accuracy: 0.6359
Epoch 3/100
19/19 [==============================] - 0s 2ms/step - loss: 0.5786 - accuracy: 0.8838
Epoch 4/100
19/19 [==============================] - 0s 997us/step - loss: 0.4877 - accuracy: 0.9214
Epoch 5/100
19/19 [==============================] - 0s 942us/step - loss: 0.4204 - accuracy: 0.9316
Epoch 6/100
19/19 [==============================] - 0s 776us/step - loss: 0.3691 - accuracy: 0.9316
Epoch 7/100
19/19 [==============================] - 0s 887us/step - loss: 0.3306 - accuracy: 0.9316
Epoch 8/100
19/19 [==============================] - 0s 942us/step - loss: 0.3018 - accuracy: 0.9316
Epoch 9/100
19/19 [==============================] - 0s 831us/step - loss: 0.2790 - accuracy: 0.9316
Epoch 10/100
19/19 [==============================] - 0s 887us/step - loss: 0.2604 - accuracy: 

In [21]:
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)


In [22]:
for target in target_list:
    y=df[target]

    #Build Dense layers for neural network
    nn_model = tf.keras.models.Sequential()
    nn_model.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=8))
    nn_model.add(tf.keras.layers.Dense(units=16, activation="relu"))
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the Sequential model together and customize metrics
    nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])


    # Train the model
    fit_model = nn_model.fit(X_scaled, y, epochs=100)

    # Evaluate the model
    model_loss, model_accuracy = nn_model.evaluate(X_scaled, y ,verbose=2)
    print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
    print(nn_model.summary())
    nn_model.save(target+"_model.h5")


Epoch 1/100
25/25 [==============================] - 0s 831us/step - loss: 0.7253 - accuracy: 0.4776
Epoch 2/100
25/25 [==============================] - 0s 873us/step - loss: 0.5595 - accuracy: 0.9091
Epoch 3/100
25/25 [==============================] - 0s 1ms/step - loss: 0.4215 - accuracy: 0.9373
Epoch 4/100
25/25 [==============================] - 0s 1ms/step - loss: 0.3212 - accuracy: 0.9373
Epoch 5/100
25/25 [==============================] - 0s 997us/step - loss: 0.2555 - accuracy: 0.9373
Epoch 6/100
25/25 [==============================] - 0s 831us/step - loss: 0.2183 - accuracy: 0.9373
Epoch 7/100
25/25 [==============================] - 0s 997us/step - loss: 0.1945 - accuracy: 0.9373
Epoch 8/100
25/25 [==============================] - 0s 790us/step - loss: 0.1756 - accuracy: 0.9398
Epoch 9/100
25/25 [==============================] - 0s 956us/step - loss: 0.1596 - accuracy: 0.9501
Epoch 10/100
25/25 [==============================] - 0s 790us/step - loss: 0.1460 - accuracy: 